# Signature Matrix Generation

This notebook documents steps used to generate a siganture matrix of scRNA-seq data for CIBERSORTx.

Revelant links:
1. The CIBERSORTx paper describing the workflow for deconvolution that uses the signature matrix: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7695353/

2. The publication with reference snRNAseq  data for nucleus accumbens: https://www.sciencedirect.com/science/article/pii/S0896627321006553

3. The link to their deposited reference data: https://github.com/LieberInstitute/10xPilot_snRNAseq-human#readme

4. Source code for Seurat function that builds a signature matrix: https://rdrr.io/github/PelzKo/dwls/man/buildSignatureMatrixUsingSeruat.html

5. An alternative R package for building signature matrices: https://github.com/ruppinlab/scSigR


## 1. Download data

The data downloaded are [processed data](https://github.com/LieberInstitute/10xPilot_snRNAseq-human?tab=readme-ov-file#processed-data). They are in the format of corresponding SingleCellExperiment R/Bioconductor objects (with reducedDims, annotations, etc.) for each of the five regions across eight donors:

-  AMY
- DLPFC
- HPC
- NAc
- sACC

In [ ]:
cd ~/project/git/sigmat/data/
wget https://libd-snrnaseq-pilot.s3.us-east-2.amazonaws.com/SCE_NAc-n8_tran-etal.rda

## 2. Data inspection

In [1]:
setwd("~/project/git/sigmat/")

library(SingleCellExperiment)
library(dplyr)
library(Seurat)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [2]:
load("./data/SCE_NAc-n8_tran-etal.rda")
ls()

[1] "PATH"         "sce.nac.tran"

In [3]:
## barcode for cells
colData(sce.nac.tran) %>% dim()
colData(sce.nac.tran) %>% colnames()
colData(sce.nac.tran) %>% rownames() %>% head()

[1] 20571    14

[1] "Barcode"          "sum"              "detected"         "doubletScore"    
 [5] "region"           "donor"            "sex"              "processBatch"    
 [9] "protocol"         "sequencer"        "sizeFactor"       "prelimCluster"   
[13] "collapsedCluster" "cellType"

[1] "donor1_AAACCCACATCGAACT.1" "donor1_AAACCCATCCAACCAA.1"
[3] "donor1_AAACGAACAATGACCT.1" "donor1_AAACGAATCACGAGGA.1"
[5] "donor1_AAACGCTAGGTTGAGC.1" "donor1_AAACGCTCACTGCATA.1"

In [4]:
## gene names
rowData(sce.nac.tran) %>% dim()
rowData(sce.nac.tran) %>% colnames()
rowData(sce.nac.tran) %>% rownames() %>% head()

[1] 33538    36

[1] "gene_id"                    "gene_version"              
 [3] "gene_name"                  "gene_source"               
 [5] "gene_biotype"               "Symbol.uniq"               
 [7] "propNucleiExprs"            "propExprsIn.Astro_A"       
 [9] "propExprsIn.Astro_B"        "propExprsIn.drop.doublet_A"
[11] "propExprsIn.drop.doublet_B" "propExprsIn.drop.doublet_C"
[13] "propExprsIn.drop.doublet_D" "propExprsIn.drop.lowNTx"   
[15] "propExprsIn.Inhib_A"        "propExprsIn.Inhib_B"       
[17] "propExprsIn.Inhib_C"        "propExprsIn.Inhib_D"       
[19] "propExprsIn.Inhib_E"        "propExprsIn.Macrophage"    
[21] "propExprsIn.Micro"          "propExprsIn.Micro_resting" 
[23] "propExprsIn.MSN.D1_A"       "propExprsIn.MSN.D1_B"      
[25] "propExprsIn.MSN.D1_C"       "propExprsIn.MSN.D1_D"      
[27] "propExprsIn.MSN.D1_E"       "propExprsIn.MSN.D1_F"      
[29] "propExprsIn.MSN.D2_A"       "propExprsIn.MSN.D2_B"      
[31] "propExprsIn.MSN.D2_C"       "propExprsIn.MSN.D2_D"      
[33] "propExprsIn.Oligo_A"        "propExprsIn.Oligo_B"       
[35] "propExprsIn.OPC"            "propExprsIn.OPC_COP"

[1] "MIR1302-2HG" "FAM138A"     "OR4F5"       "AL627309.1"  "AL627309.3" 
[6] "AL627309.2"

In [5]:
## cell types
colData(sce.nac.tran)$cellType %>% unique()
colData(sce.nac.tran)$cellType %>% unique() %>% length()

[1] Oligo_B        Micro          Astro_B        MSN.D1_A       Oligo_A       
 [6] MSN.D1_D       MSN.D2_A       drop.lowNTx    OPC            MSN.D2_B      
[11] Inhib_A        drop.doublet_A Astro_A        MSN.D1_E       MSN.D2_C      
[16] drop.doublet_B MSN.D1_C       MSN.D1_B       Inhib_D        MSN.D2_D      
[21] Macrophage     Inhib_C        MSN.D1_F       Micro_resting  Inhib_E       
[26] drop.doublet_D OPC_COP        Inhib_B        drop.doublet_C
29 Levels: Astro_A Astro_B drop.doublet_A drop.doublet_B ... OPC_COP

[1] 29

In [6]:
# single cell expression matrix
assayNames(sce.nac.tran)

[1] "counts"    "logcounts"

In [7]:
assay(sce.nac.tran, "logcounts") %>% head()

  [[ suppressing 34 column names ‘donor1_AAACCCACATCGAACT.1’, ‘donor1_AAACCCATCCAACCAA.1’, ‘donor1_AAACGAACAATGACCT.1’ ... ]]



6 x 20571 sparse Matrix of class "dgCMatrix"
                                                                               
MIR1302-2HG . . . . . . . . . . . . . . . . . . . . .         . .         . . .
FAM138A     . . . . . . . . . . . . . . . . . . . . .         . .         . . .
OR4F5       . . . . . . . . . . . . . . . . . . . . .         . .         . . .
AL627309.1  . . . . . . . . . . . . . . . . . . . . 0.1875968 . 0.6843966 . . .
AL627309.3  . . . . . . . . . . . . . . . . . . . . .         . .         . . .
AL627309.2  . . . . . . . . . . . . . . . . . . . . .         . .         . . .
                                          
MIR1302-2HG . . .         . . . . . ......
FAM138A     . . .         . . . . . ......
OR4F5       . . .         . . . . . ......
AL627309.1  . . 0.9035047 . . . . . ......
AL627309.3  . . .         . . . . . ......
AL627309.2  . . .         . . . . . ......

 .....suppressing 20537 columns in show(); maybe adjust options(max.print=, width=)
 ........

## 3. Convert to Seurat Object

In [8]:
# convert SCE to Seurat object
scdata_nac <- as.Seurat(sce.nac.tran)

# set cell types
Idents(scdata_nac) <- scdata_nac@meta.data$cellType
cell_types <- scdata_nac@meta.data$cellType %>% unique() %>% as.character()

cell_types

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from PCA_corrected_ to PCAcorrected_”
Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from PCA_opt_ to PCAopt_”


[1] "Oligo_B"        "Micro"          "Astro_B"        "MSN.D1_A"      
 [5] "Oligo_A"        "MSN.D1_D"       "MSN.D2_A"       "drop.lowNTx"   
 [9] "OPC"            "MSN.D2_B"       "Inhib_A"        "drop.doublet_A"
[13] "Astro_A"        "MSN.D1_E"       "MSN.D2_C"       "drop.doublet_B"
[17] "MSN.D1_C"       "MSN.D1_B"       "Inhib_D"        "MSN.D2_D"      
[21] "Macrophage"     "Inhib_C"        "MSN.D1_F"       "Micro_resting" 
[25] "Inhib_E"        "drop.doublet_D" "OPC_COP"        "Inhib_B"       
[29] "drop.doublet_C"

In [9]:
# check how many cells are annotated with "drop.xxx" and need to be removed
grep("drop", scdata_nac@meta.data$cellType) %>% length()

[1] 679

In [10]:
# remove unwanted cell types
unwanted_cell_types <- c("drop.lowNTx", "drop.doublet_A", "drop.doublet_B", "drop.doublet_D", "drop.doublet_C")
scdata_nac_filtered <- subset(scdata_nac, idents = setdiff(Idents(scdata_nac), unwanted_cell_types))

# set cell types for filtered data
Idents(scdata_nac_filtered) <- scdata_nac_filtered@meta.data$cellType
cell_types_filtered <- scdata_nac_filtered@meta.data$cellType %>% unique() %>% as.character()
cell_types_filtered

[1] "Oligo_B"       "Micro"         "Astro_B"       "MSN.D1_A"     
 [5] "Oligo_A"       "MSN.D1_D"      "MSN.D2_A"      "OPC"          
 [9] "MSN.D2_B"      "Inhib_A"       "Astro_A"       "MSN.D1_E"     
[13] "MSN.D2_C"      "MSN.D1_C"      "MSN.D1_B"      "Inhib_D"      
[17] "MSN.D2_D"      "Macrophage"    "Inhib_C"       "MSN.D1_F"     
[21] "Micro_resting" "Inhib_E"       "OPC_COP"       "Inhib_B"

In [11]:
# retrieve expression matrix for filtered data
scdata_nac_mat_filtered <- GetAssayData(scdata_nac_filtered, assay = "originalexp", slot = "data")
colnames(scdata_nac_mat_filtered) <- scdata_nac_filtered@meta.data$cellType

head(scdata_nac_mat_filtered)

Warning message:
“The `slot` argument of `GetAssayData()` is deprecated as of SeuratObject 5.0.0.
ℹ Please use the `layer` argument instead.”
  [[ suppressing 34 column names ‘Oligo_B’, ‘Oligo_B’, ‘Oligo_B’ ... ]]



6 x 19892 sparse Matrix of class "dgCMatrix"
                                                                               
MIR1302-2HG . . . . . . . . . . . . . . . . . . . . .         . .         . . .
FAM138A     . . . . . . . . . . . . . . . . . . . . .         . .         . . .
OR4F5       . . . . . . . . . . . . . . . . . . . . .         . .         . . .
AL627309.1  . . . . . . . . . . . . . . . . . . . . 0.1875968 . 0.6843966 . . .
AL627309.3  . . . . . . . . . . . . . . . . . . . . .         . .         . . .
AL627309.2  . . . . . . . . . . . . . . . . . . . . .         . .         . . .
                                          
MIR1302-2HG . .         . . . . . . ......
FAM138A     . .         . . . . . . ......
OR4F5       . .         . . . . . . ......
AL627309.1  . 0.9035047 . . . . . . ......
AL627309.3  . .         . . . . . . ......
AL627309.2  . .         . . . . . . ......

 .....suppressing 19858 columns in show(); maybe adjust options(max.print=, width=)
 ........

## 4. Sigature Matrix Generation for NAc region

The code is adapted from: https://github.com/PelzKo/dwls/blob/master/R/functions.R based on `buildSignatureMatrixUsingSeurat` and `DEAnalysis` functions.

In [ ]:
## Performing DE analysis using Seurat
##
## scdata: The single cell data matrix
## cell_types: A vector of the cell type annotations
##
## return: List with the differentially expressed genes for each cell type

DEAnalysis <- function(scdata, cell_types) {
    list.DE.group <- as.list(rep(0, length(cell_types)))
    for (c in cell_types) {
        de_group <- Seurat::FindMarkers(object = scdata,
                                        ident.1 = c,
                                        ident.2 = NULL,
                                        only.pos = TRUE,
                                        test.use = "bimod")

        index <- which(cell_types == c)
        list.DE.group[[index]] <- de_group
    }
    
    return(list.DE.group)
}

In [ ]:
## Building the signature matrix using Seurat
##
## scdata: The single cell data matrix
## cell_types: A vector of the cell type annotations
## diff.cutoff: The FC cutoff
## pval.cutoff: The p-value cutoff
##
## return: Computed signature matrix

buildSignatureMatrixUsingSeurat <- function(scdata,
                                            cell_types,
                                            diff.cutoff = 0.5,
                                            pval.cutoff = 0.01){
    # perform differential expression analysis
    list.DE.groups <- DEAnalysis(scdata, cell_types)
    saveRDS(list.DE.groups, "~/project/git/sigmat/data/list.DE.group.rds")
    # list.DE.group <- readRDS("~/project/git/sigmat/data/list.DE.group.rds")
    
    num_genes <- c()
    for (c in cell_types) {
        index <- which(cell_types == c)
        de_group <- list.DE.group[[index]]

        DEGenes <- rownames(de_group)[intersect(
            which(de_group$p_val_adj < pval.cutoff),
            which(de_group$avg_log2FC > diff.cutoff)
          )]

        ## why removing those genes ?
        nonMir <- grep("MIR|Mir", DEGenes, invert = TRUE)
        assign(
          paste("cluster_lrTest.table.", c, sep = ""),
          de_group[which(rownames(de_group) %in% DEGenes[nonMir]), ]
        )
        num_genes <- c(num_genes, length(DEGenes[nonMir]))
    }
    
    conditionNumbers <- c()
    for (g in 50:200) {
        Genes <- c()
        j <- 1
        for (i in cell_types) {
          if (num_genes[j] > 0) {
            temp <- paste("cluster_lrTest.table.", i, sep = "")
            temp <- get(temp)
            temp <- temp[order(temp$avg_log2FC, decreasing = TRUE), ]
            Genes <-
              c(Genes, (rownames(temp)[1:min(g, num_genes[j])]))
          }
          j <- j + 1
        }

        ## make signature matrix
        ExprSubset <- scdata_nac_mat[Genes, , drop = FALSE]
        Sig <- NULL
        for (i in as.list(cell_types)) {
          Sig <-
            cbind(Sig, (apply(ExprSubset, 1, function(y) {
              mean(y[names(y) == i])
            })))
        }
        colnames(Sig) <- cell_types
        conditionNumbers <- c(conditionNumbers, kappa(Sig))
    }
    
    # g is optimal gene number
    g <- which.min(conditionNumbers) + min(49, num_genes - 1)
    Genes <- c()
    j <- 1
    for (i in cell_types) {
        if (num_genes[j] > 0) {
          temp <- paste("cluster_lrTest.table.", i, sep = "")
          temp <- get(temp)
          temp <- temp[order(temp$avg_log2FC, decreasing = TRUE), ]
          Genes <-
            c(Genes, (rownames(temp)[1:min(g, num_genes[j])]))
        }
        j <- j + 1
    }

    # Genes <- unique(Genes)
    # Genes <- gsub("-ENSG", "_ENSG", Genes)
    ExprSubset <- scdata_nac_mat[Genes, , drop = FALSE]
    Sig <- NULL
    for (i in as.list(cell_types)) {
      Sig <-
        cbind(Sig, (apply(ExprSubset, 1, function(y) {
          mean(y[names(y) == i])
        })))
    }

    colnames(Sig) <- cell_types
    rownames(Sig) <- Genes
    
    return(Sig)
}

In [ ]:
# Build signature matrix using the filtered data
sigmat <- buildSignatureMatrixUsingSeurat(scdata_nac_filtered, cell_types_filtered)